# Coding a real face recognition system (Find a Match)

You can feel free to try this out with your own photos. But for pictures of known people, you wanna make sure that:
1. There's only one person in the picture
2. They are facing the camera
3. Clearly visible
4. That there's reasonably good lighting. 

If you use lower-quality images, the results won't be as accurate. 

In [2]:
from PIL import Image, ImageDraw
import PIL
import os
import face_recognition # access to the face detection model in DLIB

In [3]:
# Load the known images (each image is loaded into an array)
image_of_person_1 = face_recognition.load_image_file("Images/person_1.jpg")
image_of_person_2 = face_recognition.load_image_file("Images/person_2.jpg")
image_of_person_3 = face_recognition.load_image_file("Images/person_3.jpg")

''' 
On the end here, just put zero in brackets to get the first result from the array. But if you do this, make 
sure your images only contain one person each. Otherwise you won't know which face you're grabbing. 
'''

# Get the face encoding of each person. This can fail if no one is found in the photo.
person_1_face_encoding = face_recognition.face_encodings(image_of_person_1)[0] # pass in the image array
person_2_face_encoding = face_recognition.face_encodings(image_of_person_2)[0]
person_3_face_encoding = face_recognition.face_encodings(image_of_person_3)[0]

'''
Since we know these images only contain one face, we can go ahead and grab just the first result of the 
function by accessing index zero.
'''

# Create a list of all known face encodings (put all three face encodings in an array)
known_face_encodings = [
    person_1_face_encoding,
    person_2_face_encoding,
    person_3_face_encoding
]

# Load the image we want to check
unknown_image = face_recognition.load_image_file("Images/unknown_8.jpg")

# Get face encodings for any people in the picture
unknown_face_encodings = face_recognition.face_encodings(unknown_image)

# There might be more than one person in the photo, so we need to loop over each face we found
for unknown_face_encoding in unknown_face_encodings:

    # Test if this unknown face encoding matches any of the three people we know (compare_faces() function)
    results = face_recognition.compare_faces(known_face_encodings, unknown_face_encoding, tolerance=0.6)

    name = "Unknown"

    if results[0]:
        name = "Person 1"
    elif results[1]:
        name = "Person 2"
    elif results[2]:
        name = "Person 3"

    print(f"Found {name} in the photo!")


Found Person 3 in the photo!


-------------------------------------------------------

## Face Recognition Model in a Different Scenario

Completed face recognition system and those have recognized three people from three different images, listed here.

In [4]:
# Load the known images
image_of_person_1 = face_recognition.load_image_file("Images/person_1.jpg")
image_of_person_2 = face_recognition.load_image_file("Images/person_2.jpg")
image_of_person_3 = face_recognition.load_image_file("Images/person_3.jpg")

# Get the face encoding of each person. This can fail if no one is found in the photo.
person_1_face_encoding = face_recognition.face_encodings(image_of_person_1)[0]
person_2_face_encoding = face_recognition.face_encodings(image_of_person_2)[0]
person_3_face_encoding = face_recognition.face_encodings(image_of_person_3)[0]

# Create a list of all known face encodings
known_face_encodings = [
    person_1_face_encoding,
    person_2_face_encoding,
    person_3_face_encoding
]

# Load the image we want to check
# unknown_image = face_recognition.load_image_file("Images/unknown_7.jpg")
unknown_image = face_recognition.load_image_file("Images/group.jpg")

# Get face encodings for any people in the picture
face_locations = face_recognition.face_locations(unknown_image, number_of_times_to_upsample=2) # scale image 2x
unknown_face_encodings = face_recognition.face_encodings(unknown_image, known_face_locations=face_locations)

'''
We can also split this out into two separate steps so that we have more control over how it works. 
We can pass these face locations into the next function using a special known face locations parameter. 

Now that we have the code for face locations as a separate step, we can customize how it works. The main 
reason we couldn't find the face in the image is because the face was too small. To over come that, we can 
enlarge the image before we look for faces in it. To do that, we'll go to the face locations function 
and here for number_of_times_to_upsample will pass in two. 
'''

# There might be more than one person in the photo, so we need to loop over each face we found
for unknown_face_encoding in unknown_face_encodings:

    # Test if this unknown face encoding matches any of the three people we know
    results = face_recognition.compare_faces(known_face_encodings, unknown_face_encoding, tolerance=0.6)

    name = "Unknown"

    if results[0]:
        name = "Person 1"
    elif results[1]:
        name = "Person 2"
    elif results[2]:
        name = "Person 3"

    print(f"Found {name} in the photo!")


Found Person 3 in the photo!
Found Person 2 in the photo!
Found Person 1 in the photo!


-----------------------------------------

## Remember that face recognition is the multi-step pipeline

First we have to find each face, then we have to find the facial features and align each face, and then we can generate facing encodings for each face. 

If any step in that pipeline fails, the result of the system won't work. In this case, our system isn't even detecting the face because it's too small for the default settings. 

Since the face is never detected, the system can't identify it. Let's close these results.

## Scaling 
With face recognition, the trade off is usually between speed and accuracy. Scaling up the image is a lot slower and uses a lot more memory, but now it finds faces that were otherwise missed. Depending on the application that you're building, you might care more about speed. For example, if you know the person will be looking directly into the camera, you can assume their face will be large enough in the image that you won't need to upsample the image at all, this will make your code run faster. 

## Building a System (Tip)
When building a face recognition system, you'll want to play around with the upscaling settings to get the best results without making your code run too slowly. So, definitely keep this tip step in mind. 